<a href="https://colab.research.google.com/github/khushijhawar/SURVEILLANCE-SPYHOLE/blob/main/Man_Woman_WebApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
my_dir = "/content/drive/My Drive/Geeksman_AISquad_Project1_KhushiJhawar/"   #folder containing trained model

import shutil
shutil.copyfile(my_dir + "export.pkl" , "export.pkl")

'export.pkl'

## **Web Application**
Creating a web app using Flask Web Application Framework

Here, flask_ngrok is being used. The flask ngrok tool helps to make Flask apps running on localhost available over the internet.

In [ ]:
!pip install flask-ngrok
%mkdir templates -p
%mkdir uploads -p

#### To make predictions we create an HTML form which takes image file from user.

In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html lang="en" dir="ltr">

<head>
  <meta charset="utf-8">
  <link rel="preconnect" href="https://fonts.googleapis.com">
  <link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
  <link href="https://fonts.googleapis.com/css2?family=Audiowide&display=swap" rel="stylesheet">
  <title>SURVEILLANCE SPYHOLE</title>
  <style>
    body {
      background-color: #ECFCFF;
      margin: 0;
    }

    h1 {
      text-align: center;
      color: #3E64FF;
      font-size: 300%;
      margin: 0 auto;
      padding: 5%;
      font-family: 'Audiowide', cursive;
    }

    h2 {
      color: #3E64FF;
    }

    hr {
      border-color: #3E64FF;
      border-style: solid;
      border-width: 2px;
      margin: 0;
    }

    .top {
      background-color: #B2FCFF;
      font-weight: bold;
    }

    .middle {
      margin: 5% auto;
      width: 50%;
      text-align: center;
    }
  </style>
</head>

<body>
  <div class="top">
    <h1>
      SURVEILLANCE SPYHOLE <BR>
      MAN - WOMAN CLASSIFIER
    </h1>
  </div>
  <hr>
  <div class="middle">
    <h2> Upload the image to predict!</h2>
    <form action="/" method="post" enctype="multipart/form-data">
      <br>
      <input type="file" name="imagefile">
      <br>
      <p><input type="submit" value="Predict image"></p>
    </form>
    <img src="{{ url_for('upload', filename=imagefile) }}" style="width: 200px">
    <p>
    <h2>
      <b> {{ prediction_text }}</b>
    </h2>
    </p>

  </div>
</body>
</html>

Writing templates/index.html


#### Creating the application.py file.

In [ ]:
%%writefile app.py

#importing required libraries
from flask import Flask, render_template, request, send_from_directory
from flask_ngrok import run_with_ngrok
from fastai.vision import *
import os


#creating the flask object
app = Flask(__name__)
run_with_ngrok(app)

app.config["UPLOAD_FOLDER"] = 'uploads/'


#loading the model
predictor = load_learner("/content/")


#creating a predictor function
#takes image path as parameter and returns the predicted class
def my_predictor(img):
  test_img =open_image(img)
  predict_class,pred_idx,outputs=predictor.predict(test_img)
  return predict_class


#creating routes
@app.route('/', methods=['GET'])
def home():
  return render_template('index.html')

@app.route('/', methods=['POST'])
def predict():
  # predicting images
  imagefile = request.files['imagefile']
  filename = imagefile.filename
  image_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
  imagefile.save(image_path)
  
  result = my_predictor(image_path)
  
  return render_template('index.html', imagefile = imagefile.filename, prediction_text='Image is of a {}'.format(result))

@app.route('/uploads/<filename>')
def upload(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

if __name__=='__main__':
  app.run()

Writing app.py


#### procfile.txt

In [ ]:
%%writefile procfile.txt
web: gunicorn app:app

Writing procfile.txt


In [ ]:
pip install gunicorn

     |████████████████████████████████| 79 kB 3.4 MB/s 


#### requirements.txt

In [ ]:
!pip freeze > requirements_temp.txt

In [ ]:
%%writefile requirements.txt
fastai==1.0.61
Flask==1.1.4
flask-ngrok==0.0.25
gunicorn==20.1.0
numpy==1.19.5
pandas==1.1.5

Writing requirements.txt
